# Project 1 (Tolled User Equilibrium)

***Chun-Chien Hsiao***

As the leading modeler you are charged to **determine the user equilibrium (UE) flow distribution** across the arterial highway network of Sioux Falls, South Dakota. You are asked to **report link volumes and point out the bottlenecks**, corresponding to the most congested links of the highway network, based on an established $\frac{v}{c}$ standard, where $\frac{v}{c} > 0.90$. Based on your UE analysis you are tasked to **propose an effective congestion pricing scheme** to be considered by the metropolitan planning organization of the Sioux Falls area in South Dakota. In order to achieve this objective, the system optimum flow distribution needs to be estimated and the marginal-cost pricing link tolls need to be presented. As a modeler you are tasked to verify that the marginal-cost pricing scheme is a first- best tolling scheme. The network configuration in Sioux Falls, South Dakota, is presented in Figure 1.

Data on the capacities $c_a$ and the free flow travel times $t_a^0$ of the network links are given in Table 1. Table 2 presents the origin-destination (OD) matrix for the Sioux Falls network. The link travel times (link performance functions) are given by the BPR function: $t_a(x) = t_a^0(1+0.15(\frac{x_a}{c_a})^4)$.

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import integrate
import math
import networkx as nx
import matplotlib.pyplot as plt

# Frank-Wolfe Algorithm

### I. Network Properties

In [2]:
# link data
dflink = pd.read_excel('Data.xlsx', index_col=0, sheet_name=0)
dflink['ca'] = dflink['ca']*1000
dflink

,ta0(minutes),ca,From,To
Link Number,,,,
1,3.6,6020.0,1,2
2,2.4,9010.0,1,3
3,3.6,12020.0,2,1
4,3.0,15920.0,2,6
5,2.4,46810.0,3,1
...,...,...,...,...
72,2.4,10000.0,23,22
73,1.2,10160.0,23,24
74,2.4,11380.0,24,13


In [3]:
# draw a network
G = nx.from_pandas_edgelist(dflink, source='From', target='To', edge_attr=('ta0(minutes)', 'ca'), create_using=nx.DiGraph)
G.add_node(1, loc=(1,8))
G.add_node(2, loc=(4,8))
G.add_node(3, loc=(1,7))
G.add_node(4, loc=(2,7))
G.add_node(5, loc=(3,7))
G.add_node(6, loc=(4,7))
G.add_node(7, loc=(5,6))
G.add_node(8, loc=(4,6))
G.add_node(9, loc=(3,6))
G.add_node(10, loc=(3,5))
G.add_node(11, loc=(2,5))
G.add_node(12, loc=(1,5))
G.add_node(13, loc=(1,1))
G.add_node(14, loc=(2,3))
G.add_node(15, loc=(3,3))
G.add_node(16, loc=(4,5))
G.add_node(17, loc=(4,4))
G.add_node(18, loc=(5,5))
G.add_node(19, loc=(4,3))
G.add_node(20, loc=(4,1))
G.add_node(21, loc=(3,1))
G.add_node(22, loc=(3,2))
G.add_node(23, loc=(2,2))
G.add_node(24, loc=(2,1))

# get the node and edge attribute
pos = nx.get_node_attributes(G, 'loc')
time = nx.get_edge_attributes(G, 'ta0(minutes)')
capacity = nx.get_edge_attributes(G, 'ca')
edgelist = list(G.edges())
nodelist = list(G.nodes())

# draw graph
# plt.figure(figsize=(6,8), dpi=100)
# nx.draw_networkx(G, pos, arrows=True, with_labels=True)
# plt.show()

In [4]:
# demand of each OD
dfnode = pd.read_excel('Data.xlsx', index_col=1, sheet_name=1, skiprows=[0])
dfnode = dfnode.drop(dfnode.columns[0], axis=1)
dfnode

,1,2,4,5,10,11,13,14,15,19,20,21,22,24
1,0,1320,1320,1320,1080,1100,1250,990,950,900,590,590,770,740
2,1320,0,1250,1300,1100,1120,900,950,940,1300,590,680,670,590
4,1320,1250,0,1320,1080,1070,950,900,840,800,1620,640,590,800
5,1320,1300,1320,0,1130,970,910,880,810,730,800,810,940,590
10,1080,1100,1080,1130,0,1330,900,990,1320,1170,950,900,970,590
11,1100,1120,1070,970,1330,0,940,1320,1110,950,740,610,1100,1050
13,1250,900,950,910,900,940,0,870,860,680,590,620,670,1320
14,990,950,900,880,990,1320,870,0,1320,1130,950,870,900,1130
15,950,940,840,810,1320,1110,860,1320,0,1320,1270,1140,1320,910
19,900,1300,800,730,1170,950,680,1130,1320,0,1320,1110,1100,800


In [5]:
# Origin and Destination
noncentral = [3,6,7,8,9,12,16,17,18,23]
origins, destinations = nodelist, nodelist

for element in noncentral:
    if element in origins:
        origins.remove(element)
    elif element in destinations:
        destinations.remove(element)

OD = [[a,b] for a in origins for b in destinations]

# Demand(Dict)
demand = {a: {b: dfnode[a][b] for b in destinations} for a in origins}

In [6]:
SO_DataFrame = pd.read_csv('SO_DataFrame.csv', index_col=0)
tollij = SO_DataFrame['Toll']
toll = {edgelist[i]: tollij[i] for i in range(len(edgelist))}

### Travel Time Function

In [7]:
def tij(t0,xa,ca,toll):
    tij = t0*(1+0.15*(xa/ca)**4)+toll
    return tij

def zij(t0,xa,ca,toll):
    zij = t0*(xa+0.03*xa**5/ca**4)+toll*xa
    return zij

### All-or-Nothing Algorithm

In [8]:
def all_nothing(timefeed):
    # Define a new graph with new feed time
    G_new = nx.DiGraph()
    G_new.add_weighted_edges_from((k,v, timefeed[(k,v)]) for k,v in edgelist)
    
    # Bellman-Ford Algorithm
    allpaths = dict(nx.all_pairs_bellman_ford_path(G_new, weight='weight'))
    
    # Clean all pairs path to OD pairs in Project 1
    for i in range(len(noncentral)):
        del allpaths[noncentral[i]]
    
    for i in range(len(origins)):
        for j in range(len(noncentral)):
            del allpaths[origins[i]][noncentral[j]]
    
    # Finding ya
    ya_iter = {(u,v): 0 for u,v in edgelist}
    
    for k in range(len(OD)):
        ODpath = allpaths[OD[k][0]][OD[k][1]]
        used_link = []
        
        if len(ODpath)>1:
            for j in range(len(ODpath)-1):
                B = []
                B.append(ODpath[j])
                B.append(ODpath[j+1])
                used_link.append(B)
        
        for i in range(len(used_link)):
            (used_link[i][0], used_link[i][1])
            ya_iter[(used_link[i][0], used_link[i][1])] += demand[OD[k][0]][OD[k][1]]
        
    return ya_iter

### Bisection Algorithm

In [9]:
def bisection(x,y,edgelist,time,capacity):
    ya_iter = []
    xa_iter = []
    ya_iter = y
    xa_iter = x
    
    a = 0
    b = 1
    
    def fa(alpha):
        fa = 0
        for i in range(len(edgelist)):
            xa = xa_iter[edgelist[i]]
            ya = ya_iter[edgelist[i]]
            change = ya - xa
            time0 = time[edgelist[i]]
            tau = toll[edgelist[i]]
            ca = capacity[edgelist[i]]
            
            fa += (time0+tau)*(xa_iter[edgelist[i]]+alpha*change)+0.03*(time0+tau)/(ca**4)*(xa+alpha*change)**5
        return fa
    
    def dfa(alpha):
        dfa = 0
        for i in range(len(edgelist)):
            xa = xa_iter[edgelist[i]]
            ya = ya_iter[edgelist[i]]
            change = ya-xa
            time0 = time[edgelist[i]]
            tau = toll[edgelist[i]]
            ca = capacity[edgelist[i]]
            
            dfa += (time0+tau)*change+0.15*(time0+tau)/(ca**4)*change*(xa+alpha*change)**4
        return dfa
    
    m = (a+b)/2
    
    while (np.abs(a-b))>=10**(-10):
        if np.sign(dfa(m))<0:
            a = m
        else:
            b = m
        m = (a+b)/2
    return m

### Initiation

In [10]:
time_int = {(u,v): time[(u,v)]+toll[(u,v)] for u,v in edgelist}
#print(time_int)
xa = all_nothing(time_int)
#print(xa)

for (u,v) in edgelist:
    xa[(u,v)] = float(xa[(u,v)])

Zij = {(u,v): zij(time[(u,v)], xa[(u,v)], capacity[(u,v)], toll[(u,v)]) for u,v in edgelist}
Z = sum(Zij.values())

print('Initialization:')
print('Objective Z = {:.4f}.'.format(Z))

Initialization:
Objective Z = 1398837.4039.


### Frank-Wolfe Iterations for TUE

In [11]:
error = 100
tol = 5*10**(-5)
xa_old = xa
noi = 0

for u,v in edgelist:
    xa_old[(u,v)] = float(xa_old[(u,v)])
    
while error>tol:
    print('Iteration: {:.0f}'.format(noi+1))
    
    # Calcualte the new travel time
    newtime = {(u,v): tij(time[(u,v)], xa_old[(u,v)], capacity[(u,v)], toll[(u,v)]) for u,v in edgelist}
    
    # All-or-Nothing Algorithm
    ya = all_nothing(newtime)
    
    for u,v in edgelist:
        ya[(u,v)] = float(ya[(u,v)])
    
    # Find Alpha
    alpha = bisection(xa_old, ya, edgelist, time, capacity)
    print('The step size of this iteration: {:.4f}'.format(alpha))
    
    # New Direction
    xa_new = {(u,v): xa_old[(u,v)]+alpha*(ya[(u,v)]-xa_old[(u,v)]) for u,v in edgelist}
    
    # Objective
    Zij = {(u,v): zij(time[(u,v)], xa_new[(u,v)], capacity[(u,v)], toll[(u,v)]) for u,v in edgelist}
    Z = sum(Zij.values())
    print('The obejctive value of this iteration: {:.4f}'.format(Z))
    
    # Error
    eij = {(u,v): (xa_old[(u,v)]-xa_new[(u,v)])**2 for u,v in edgelist}
    error = np.sqrt(sum(eij.values()))/sum(xa_old.values())
    print('The error of this iteration: {:.5f}'.format(error))
    
    if error>=tol:
        print('The error is still too large, please keep going!\n')
    else:
        print('The error is within the tolerance, you\'re all set!\n')
    
    # Update Flow
    xa_old = xa_new
    for u,v in edgelist:
        xa_old[(u,v)] = float(xa_old[(u,v)])
    noi += 1

Iteration: 1
The step size of this iteration: 0.4156
The obejctive value of this iteration: 1397148.4922
The error of this iteration: 0.02196
The error is still too large, please keep going!

Iteration: 2
The step size of this iteration: 0.4074
The obejctive value of this iteration: 1396452.2887
The error of this iteration: 0.00948
The error is still too large, please keep going!

Iteration: 3
The step size of this iteration: 0.2364
The obejctive value of this iteration: 1396065.4783
The error of this iteration: 0.00626
The error is still too large, please keep going!

Iteration: 4
The step size of this iteration: 0.4963
The obejctive value of this iteration: 1395931.3834
The error of this iteration: 0.01025
The error is still too large, please keep going!

Iteration: 5
The step size of this iteration: 0.1211
The obejctive value of this iteration: 1395709.0501
The error of this iteration: 0.00366
The error is still too large, please keep going!

Iteration: 6
The step size of this itera

In [12]:
def tsij(t0, xa, ca):
    tsij = t0*(1+0.15*(xa/ca)**4)*xa
    return tsij

ue_travel_time = {(u,v): tsij(time[(u,v)], xa_new[(u,v)], capacity[(u,v)]) for (u,v) in edgelist}
UE_travel_time = sum(ue_travel_time.values())

print('Final Outcome:')
print('Objetive: {:.5f}'.format(Z))
print('Total Travel Time: {:.5f}'.format(UE_travel_time))
print('Eventual Error: {:.10f}'.format(error))
print('Number of Iterations: {:.0f}'.format(noi))

Final Outcome:
Objetive: 1395703.19202
Total Travel Time: 1255810.00242
Eventual Error: 0.0000000000
Number of Iterations: 7


In [13]:
TUE_DataFrame = pd.DataFrame.from_dict(xa_new, orient='index', columns=['TUE Flow'])
TUE_DataFrame['Time0'] = time.values()
TUE_DataFrame['Capacity'] = capacity.values()
TUE_DataFrame['TUE Travel Time'] = TUE_DataFrame['Time0']*(1+0.15*(TUE_DataFrame['TUE Flow']/TUE_DataFrame['Capacity'])**4)
TUE_DataFrame['V/C'] = TUE_DataFrame['TUE Flow']/TUE_DataFrame['Capacity'] 
TUE_DataFrame = TUE_DataFrame[['Time0', 'Capacity', 'TUE Flow', 'TUE Travel Time', 'V/C']]
TUE_DataFrame

,Time0,Capacity,TUE Flow,TUE Travel Time,V/C
"(1, 2)",3.6,6020.0,5626.797772,4.012148,0.934684
"(1, 3)",2.4,9010.0,10273.202228,3.008453,1.140200
"(2, 1)",3.6,12020.0,2983.480701,3.602050,0.248210
"(2, 6)",3.0,15920.0,12716.797772,3.183211,0.798794
"(3, 1)",2.4,46810.0,12916.519299,2.402087,0.275935
...,...,...,...,...,...
"(20, 21)",3.6,10120.0,5908.076329,3.662727,0.583802
"(20, 22)",3.0,10150.0,4460.009703,3.016776,0.439410
"(21, 20)",3.6,10120.0,5359.670961,3.642484,0.529612
"(21, 22)",1.2,10460.0,7004.885119,1.236203,0.669683


In [14]:
Bottleneck = TUE_DataFrame[(TUE_DataFrame['V/C']>=0.9)]
Bottleneck

,Time0,Capacity,TUE Flow,TUE Travel Time,V/C
"(1, 2)",3.6,6020.0,5626.797772,4.012148,0.934684
"(1, 3)",2.4,9010.0,10273.202228,3.008453,1.140200
"(6, 2)",3.0,9920.0,10073.480701,3.478502,1.015472
"(11, 14)",2.4,9750.0,8837.702429,2.643020,0.906431
"(8, 6)",1.2,9800.0,8875.307069,1.321088,0.905644
"(14, 11)",2.4,9750.0,8981.323807,2.659206,0.921161
"(13, 24)",2.4,10180.0,10330.000000,2.781692,1.014735
"(24, 13)",2.4,11380.0,10669.995666,2.678221,0.937609


### SO v.s TUE

In [15]:
SO = pd.read_csv('SO_DataFrame.csv', index_col=0)
TUE = pd.read_csv('TUE_DataFrame.csv', index_col=0)

In [16]:
data = {'SO Flow': SO['SO Flow'],'TUE Flow': TUE['TUE Flow'], 'SO Travel Time': SO['SO Travel Time'], 'TUE Travel Time': TUE['TUE Travel Time']}
DF = pd.DataFrame(data=data)
DF['Flow Diff (%)'] = (DF['TUE Flow']-DF['SO Flow'])/DF['SO Flow']*100
DF['Travel Time Diff (%)'] = (DF['TUE Travel Time']-DF['SO Travel Time'])/DF['SO Travel Time']*100
DF = DF[['SO Flow', 'TUE Flow', 'Flow Diff (%)', 'SO Travel Time', 'TUE Travel Time', 'Travel Time Diff (%)']]
DF

,SO Flow,TUE Flow,Flow Diff (%),SO Travel Time,TUE Travel Time,Travel Time Diff (%)
"(1, 2)",5513.90,5591.719621,1.411335,3.98,4.001966,0.551910
"(1, 3)",10387.79,10308.280379,-0.765414,3.04,3.016805,-0.762977
"(2, 1)",3091.37,3019.669681,-2.319370,3.60,3.602151,0.059746
"(2, 6)",12602.21,12681.719621,0.630918,3.18,3.181198,0.037666
"(3, 1)",12810.31,12880.330319,0.546593,2.40,2.402064,0.085989
...,...,...,...,...,...,...
"(20, 21)",6116.01,5895.785332,-3.600790,3.67,3.662207,-0.212347
"(20, 22)",4217.42,4465.045527,5.871493,3.01,3.016852,0.227642
"(21, 20)",5755.95,5189.454617,-9.841909,3.66,3.637339,-0.619163
"(21, 22)",6958.76,7313.241416,5.094031,1.24,1.243012,0.242869


In [17]:
DF.to_csv('SO v.s. TUE.csv')